In [1]:
import requests
import json
import pandas as pd
import time
from dateutil.parser import parse
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
#필수 함수 정의
apikey = 'X1o3cmgN7nsBDsInu4km3XtlPD051oPC'
wordType = 'matche'
main_url = 'https://api.neople.co.kr/cy/'
player_url = 'https://api.neople.co.kr/cy/players/'
matches_url = 'https://api.neople.co.kr/cy/matches/'
pd.options.display.max_columns = 300
columnsname = ['nickname','characterName','position','attribute1','attribute2','attribute3','hand','hand(rarityName)','head','head(rarityName)','chest','chest(rarityName)','waist','waist(rarityName)','leg','leg(rarityName)','foot','foot(rarityName)','accessories1','accessories1(rarityName)','accessories2','accessories2(rarityName)','recovery','recovery(rarityName)','accelerator','accelerator(rarityName)','attack','attack(rarityName)','defense','defense(rarityName)','special','special(rarityName)','neck','neck(rarityName)','accessories3','accessories3(rarityName)','accessories4','accessories4(rarityName)','random','level','killCount','deathCount','assistCount','attackPoint','damagePoint','battlePoint','sightPoint','getCoin','spendConsumablesCoin','sentinelKillCount','demolisherKillCount','playTime','minLifeTime','maxLifeTime','responseTime','team','matchId','map']

In [2]:
# 데이터 수집단 함수 정리
def get_playerid(nickname):#플레이어 고유 id를 구하는 함수
    api = main_url+'players?nickname='+nickname+'&wordType='+wordType+'&apikey='+apikey
    playerdata = requests.get(api)
    playerId = playerdata.json()['rows'][0]['playerId']
    return playerId

#매치 id 가져오기
def get_matches(playerId):
    matchids = []
    gameTypeId = 'rating'
    #검색 시작일
    startDate='2022-01-01'
    endDate='2022-03-30'
    limit = 1000
    player_url = 'https://api.neople.co.kr/cy/players/'
    cc = player_url+playerId+'/matches?gameTypeId='+gameTypeId+'&apikey='+apikey+'&startDate='+startDate+'&endDate='+endDate
    playdata = requests.get(cc)
    playdata_info= playdata.json()
    while True:
        try:
            for matche in playdata_info['matches']['rows']:
                matchids.append(matche['matchId'])
            next = playdata_info['matches']['next']
            cc_next = player_url+playerId+'/matches?gameTypeId='+gameTypeId+'&apikey='+apikey+'&startDate='+startDate+'&endDate='+endDate+'&next='+next    
            playdata = requests.get(cc_next)
            playdata_info= playdata.json()
            time.sleep(0.5)
        except:
            break
        
    return matchids


def win_and_lose(matchId):#매치 id를 들고와서 승리팀 조합과,패배팀 조합으로 나눠줌
    api3=matches_url+matchId+'?&apikey='+apikey
    game = requests.get(api3)
    lose_team = []
    win_team = []
    lose_ch = []
    win_ch = []
    try:
        if game.json()['teams'][0]['result'] == 'lose':
            for lose_player in game.json()['teams'][0]['players']:
                lose_team.append(lose_player)
            for win_player in game.json()['teams'][1]['players']:
                win_team.append(win_player)
        elif game.json()['teams'][0]['result'] == 'win':
            for lose_player in game.json()['teams'][1]['players']:
                lose_team.append(lose_player)
            for win_player in game.json()['teams'][0]['players']:
                win_team.append(win_player)
        lose_ch =[]
        win_ch = []
        for i in range(0,10):
            if game.json()['players'][i]['playerId'] in lose_team:
                lose_ch.append(game.json()['players'][i]['playInfo']['characterName'])
            elif game.json()['players'][i]['playerId'] in win_team:
                win_ch.append(game.json()['players'][i]['playInfo']['characterName'])
    except:pass
    return win_ch,lose_ch

#중복매칭 데이터 방지를 위한 데이터 불러오기
def road_antdpData(graed):
    ant_data = []
    antdpData = pd.read_csv('MatcesData/{0}_antdpData.csv'.format(graed))
    #csv를 리스트로 변환
    for v in antdpData['0']:
        ant_data.append(v)
    return ant_data

# 이번에 얻은 데이터를 중복방지 데이터에 기록하며 저장
def antdpData_save(antdpData,now_matcs,graed):
    antdpData.extend(now_matcs)
    antdpData = pd.DataFrame(antdpData)
    antdpData.to_csv('MatcesData/{0}_antdpData.csv'.format(graed),index=False, encoding="utf-8-sig")
    
#닉네임 수집기
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
#range 검색할 페이지 범위 1부터 200전까지
def get_nicknames(graed,start,end):
    names = []
    if start > 0:
        for i in range(start,end):
            rank = requests.get('https://cyphers.nexon.com/cyphers/article/ranking/total/20/{0}'.format(i))
            rank =rank.text
            soup1 = BeautifulSoup(rank,"html.parser")
            a = soup1.find_all(class_='non_list')
            #non_list라는 태그가 존재할경우 반복문을 끝낸다(없는 페이지기 때문)
            if len(a) > 0:
                break
            soup1 = soup1.findAll(target="_blank")
            for i in soup1:
                    names.append(i.text)#.text로 태그부분 제거
        #들어가면 안되는 요소
        trash = ['회사소개','넥슨PC방','오픈 API','']
        #trash부분을 삭제
        names = [i for i in names if i not in trash]
        # 위식가 밑식은 동일하다
        # for i in names:
        #     if i not in trash:
        #         names.append(i)
        pd.DataFrame(names).to_csv('{0}nickname.csv'.format(graed),index=False, encoding="utf-8-sig")
        return names
    else:
        print('start값이 이상합니다.')
        
def road_nicknames(graed):
    names = []
    antdpData = pd.read_csv('{0}nickname.csv'.format(graed),encoding="utf-8-sig")
    #csv를 리스트로 변환
    for v in antdpData['0']:
        names.append(v)
    return names

In [3]:
#데이터 수집 상세단
def ingame_data(matchId):
    api3=matches_url+matchId+'?&apikey='+apikey
    game = requests.get(api3).json()
    basic = pd.DataFrame(columns=columnsname)
    win_ch,lose_ch = win_and_lose(matchId)
    try:
        for i in range(0,10):
            ch = []
            if game['players'][i]['playInfo']['characterName'] in lose_ch:
                ch.append(game['players'][i]['nickname'])
                ch.append(game['players'][i]['playInfo']['characterName'])
                ch.append(game['players'][i]['position']['name'])
                for j in range(0,3):
                    if j == 0:
                        ch.append(game['players'][i]['position']['attribute'][j]['name'])
                    if j == 1:
                        ch.append(game['players'][i]['position']['attribute'][j]['name'])
                    if j == 2:
                        ch.append(game['players'][i]['position']['attribute'][j]['name'])
                for k in range(0,16):
                    ch.append(game['players'][i]['items'][k]['itemName'])
                    ch.append(game['players'][i]['items'][k]['rarityName'])
                ch.append(game['players'][i]['playInfo']['random']) # 랜덤 유무
                ch.append(game['players'][i]['playInfo']['level'])#레벨
                ch.append(game['players'][i]['playInfo']['killCount'])#죽인 횟수
                ch.append(game['players'][i]['playInfo']['deathCount'])#죽은 횟수
                ch.append(game['players'][i]['playInfo']['assistCount'])#어시
                ch.append(game['players'][i]['playInfo']['attackPoint'])#준 데미지
                ch.append(game['players'][i]['playInfo']['damagePoint'])#받은 데미지
                ch.append(game['players'][i]['playInfo']['battlePoint'])#전투점수
                ch.append(game['players'][i]['playInfo']['sightPoint'])#시야점수
                ch.append(game['players'][i]['playInfo']['getCoin'])#흭득 코인
                ch.append(game['players'][i]['playInfo']['spendConsumablesCoin']) # 소모품 코인
                ch.append(game['players'][i]['playInfo']['sentinelKillCount']) # 센티널(중립) 코인
                ch.append(game['players'][i]['playInfo']['demolisherKillCount']) # 철거반 코인
                ch.append(game['players'][i]['playInfo']['playTime'])
                ch.append(game['players'][i]['playInfo']['minLifeTime'])
                ch.append(game['players'][i]['playInfo']['maxLifeTime'])
                ch.append(game['players'][i]['playInfo']['responseTime'])
                ch.append(0)
                ch.append(matchId)
                ch.append(game['map']['name'])
                ch = pd.DataFrame(ch)
                ch = ch.transpose()
                ch.columns = columnsname
                basic = pd.concat([basic,ch])
        for i in range(0,10):
            ch = []
            if game['players'][i]['playInfo']['characterName'] in win_ch:
                ch.append(game['players'][i]['nickname'])
                ch.append(game['players'][i]['playInfo']['characterName'])
                ch.append(game['players'][i]['position']['name'])
                for j in range(0,3):
                    if j == 0:
                        ch.append(game['players'][i]['position']['attribute'][j]['name'])
                    if j == 1:
                        ch.append(game['players'][i]['position']['attribute'][j]['name'])
                    if j == 2:
                        ch.append(game['players'][i]['position']['attribute'][j]['name'])
                for k in range(0,16):
                    ch.append(game['players'][i]['items'][k]['itemName'])
                    ch.append(game['players'][i]['items'][k]['rarityName'])
                ch.append(game['players'][i]['playInfo']['random']) # 랜덤 유무
                ch.append(game['players'][i]['playInfo']['level'])#레벨
                ch.append(game['players'][i]['playInfo']['killCount'])#죽인 횟수
                ch.append(game['players'][i]['playInfo']['deathCount'])#죽은 횟수
                ch.append(game['players'][i]['playInfo']['assistCount'])#어시
                ch.append(game['players'][i]['playInfo']['attackPoint'])#준 데미지
                ch.append(game['players'][i]['playInfo']['damagePoint'])#받은 데미지
                ch.append(game['players'][i]['playInfo']['battlePoint'])#전투점수
                ch.append(game['players'][i]['playInfo']['sightPoint'])#시야점수
                ch.append(game['players'][i]['playInfo']['getCoin'])#흭득 코인
                ch.append(game['players'][i]['playInfo']['spendConsumablesCoin']) # 소모품 코인
                ch.append(game['players'][i]['playInfo']['sentinelKillCount']) # 센티널(중립) 코인
                ch.append(game['players'][i]['playInfo']['demolisherKillCount']) # 철거반 코인
                ch.append(game['players'][i]['playInfo']['playTime'])
                ch.append(game['players'][i]['playInfo']['minLifeTime'])
                ch.append(game['players'][i]['playInfo']['maxLifeTime'])
                ch.append(game['players'][i]['playInfo']['responseTime'])
                ch.append(1)
                ch.append(matchId)
                ch.append(game['map']['name'])
                ch = pd.DataFrame(ch)
                ch = ch.transpose()
                ch.columns = columnsname
                basic = pd.concat([basic,ch])
    except:
        pass
    return basic

In [4]:
filename = 'low'
#하위 유저를 크롤링하는 함수 
#크롤링 함수를 사용
users = get_nicknames(filename,99,100)#'저장할이름,시작페이지,종료페이지'
basic = pd.DataFrame(columns=columnsname) #분석할 데이터프레임 생성
# 크롤링한 아이디를 바탕으로 매칭id를 불러옴
users_matcs = []
for nickname in users:
    playerId = get_playerid(nickname)
    matchids = get_matches(playerId)
    antdpData = road_antdpData(filename)
    now_matcs = []
    for v in matchids:
        if v not in antdpData:
            now_matcs.append(v)
            users_matcs.append(v)
    antdpData_save(antdpData,now_matcs,filename)
#불러온 매칭id를 바탕으로 게임속 데이터를 csv로 저장
for matchId in users_matcs: #
    b = ingame_data(matchId)
    basic = pd.concat([basic,b])
    time.sleep(0.3)
    basic = basic.reset_index(drop=True)
    basic.to_csv('gamedata/{}.csv'.format(name),index=False, encoding="utf-8-sig")#저장이름 설정

KeyboardInterrupt: 

In [5]:
lowuser = pd.read_csv('gamedata/low.csv')
topuser = pd.read_csv('gamedata/top.csv')

In [6]:
#탈주있는 게임 삭제
value_counts = topuser.matchId.value_counts()
error_matchs = value_counts[value_counts != 10].index
error_matchs_id = []
for a in error_matchs:
    error_matchs_id.append(a)
error_index = []
for error_id in error_matchs_id:
    indexs = topuser[topuser.matchId==error_id].index
    for i in indexs:
         topuser = topuser.drop(i)
topuser = topuser.reset_index(drop=True)

In [7]:
topuser[topuser.team == 1].characterName.value_counts()

아이작     268
론       268
엘프리데    229
드렉슬러    211
플로리안    187
       ... 
샬럿       37
클리브      26
탄야       24
미쉘       21
빅터        8
Name: characterName, Length: 73, dtype: int64

In [9]:
topuser[topuser.team == 0].characterName.value_counts()

엘프리데    246
아이작     211
드렉슬러    203
론       203
휴톤      189
       ... 
히카르도     34
탄야       28
클리브      26
미쉘       19
빅터        8
Name: characterName, Length: 73, dtype: int64

In [107]:
topuser.to_csv('FinalData/topuser.csv'.format(name),index=False, encoding="utf-8-sig")

,nickname,characterName,position,attribute1,attribute2,attribute3,hand,hand(rarityName),head,head(rarityName),chest,chest(rarityName),waist,waist(rarityName),leg,leg(rarityName),foot,foot(rarityName),accessories1,accessories1(rarityName),accessories2,accessories2(rarityName),recovery,recovery(rarityName),accelerator,accelerator(rarityName),attack,attack(rarityName),defense,defense(rarityName),special,special(rarityName),neck,neck(rarityName),accessories3,accessories3(rarityName),accessories4,accessories4(rarityName),random,level,killCount,deathCount,assistCount,attackPoint,damagePoint,battlePoint,sightPoint,getCoin,spendConsumablesCoin,sentinelKillCount,demolisherKillCount,playTime,minLifeTime,maxLifeTime,responseTime,team,matchId,map
0,최두나,제이,탱커,전투 태세,할인 판매,최후의 저항,E 패스티스트 건,유니크,E 퍼니시먼트,유니크,E 헤븐즈 도어,유니크,E 트러블 메이커,유니크,E 데스페라도,유니크,S 퀵 앤 데드,유니크,SB스프리건 루인,레어,니들 터퀴즈,레어,퍼니시 스파클링,레어,런업 스프린터,레어,퍼니시 파이크,레어,솔리드 스위퍼,레어,PS 레이더 R4,언커먼,E 멕시칸 불독,유니크,난사 레그람,레어,타임투킬 레그람,레어,False,48,2,6,18,24769,39299,193,292,15370,3510,16,11,1076,58,305,174,0,f9fa9cd7fd0fe943b6e67ab6d67fe914cf7afdff5267bb...,메트로폴리스
1,도도코대마왕,엘프리데,원거리딜러,충동 구매,할인 판매,준비된 한 발,E 결정적 순간,유니크,S 감춰진 시선,유니크,E 오버 필링,유니크,E 레스트레인트,유니크,S 완벽한 자유,유니크,S 온화한 발걸음,유니크,팰리스 메모,레어,저스티스 페인트북,레어,체스터 더블버거,레어,페이트 엑셀,레어,페이트 파이크,레어,솔리드 스위퍼,레어,타나토스 맥시머,레어,E 잔혹한 처벌,유니크,SB데블스 데브,레어,정해진 운명,유니크,False,52,3,7,12,43365,21944,131,158,16770,2730,17,16,1076,25,304,237,0,f9fa9cd7fd0fe943b6e67ab6d67fe914cf7afdff5267bb...,메트로폴리스
2,참지않는사람,이글,탱커,전투 태세,할인 판매,최후의 저항,E 브리닝 슈나이더,유니크,E 스크라이 쇼크,유니크,S 언밸런스 레퀴엠,유니크,E 화이트 샤르페,유니크,S 킬 블레이드,유니크,E 바이스 슈트름,유니크,장작 레그람,레어,사영 레그람,레어,첼시 콜라 MAX,레어,리세스 스프린터,레어,스네이크 파이크,레어,솔리드 스위퍼,레어,PS 레이더 R4,언커먼,E 스네이크 오빗,유니크,초승달 셀레네,레어,쉬베르트 바이스,유니크,False,61,7,6,10,41002,33870,242,281,18450,3620,47,16,1076,39,267,206,0,f9fa9cd7fd0fe943b6e67ab6d67fe914cf7afdff5267bb...,메트로폴리스
3,배달요원,미아,원거리딜러,완벽주의자,전장의 학살자,완전한 성장,E 이볼브 가이아,유니크,E 디바인 그레이스,유니크,E 옐로우 루트웨이브,유니크,E 프릭클 에린,유니크,E 와일드 크로스,유니크,E 이네이쳐 게브,유니크,도장 바이리프,레어,가시 바이리프,레어,체스터 더블버거,레어,런업 스프린터,레어,우드 파이크,레어,솔리드 스위퍼,레어,세계수 맥시머,레어,S 레드 크로니클,유니크,뿌리 바이리프,레어,트리콜로 트와인,유니크,False,48,3,9,8,17360,27630,124,167,15610,3300,15,17,1076,10,266,276,0,f9fa9cd7fd0fe943b6e67ab6d67fe914cf7afdff5267bb...,메트로폴리스
4,용1현,티모시,탱커,최후의 저항,충동 구매,전투 태세,DB애쉬 마노,레어,S 이너 스트랭스,유니크,E 레클러메이션,유니크,E 뉴 비기닝,유니크,S 오소틱,유니크,S 내딛은 걸음,유니크,번 버스터 레그람,레어,잿불 레그람,레어,애쉬 스파클링,레어,런업 스프린터,레어,애쉬 파이크,레어,애쉬 타즈,레어,PS 레이더 R4,언커먼,S 달콤한 죄책감,유니크,화염포 레그람,레어,라스트 릭,유니크,False,40,6,7,5,30869,38027,240,328,14670,5200,5,3,1076,37,300,135,0,f9fa9cd7fd0fe943b6e67ab6d67fe914cf7afdff5267bb...,메트로폴리스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13365,케니스원툴,티모시,탱커,최후의 저항,낙법,불굴의 의지,E 덱스트러스,유니크,S 이너 스트랭스,유니크,E 레클러메이션,유니크,E 뉴 비기닝,유니크,S 오소틱,유니크,S 내딛은 걸음,유니크,번 버스터 레그람,레어,잿불 레그람,레어,애쉬 스파클링,레어,런업 스프린터,레어,애쉬 파이크,레어,애쉬 플래쉬,레어,인사이트 엔진T7,언커먼,도전의 로사리오,레어,화염포 펜슬,레어,라스트 릭,유니크,False,41,7,5,16,25114,45088,322,432,16000,5000,19,3,1260,45,474,102,1,7e5c628d939498bc3ee75fb96026f9aef96197ce71da93...,메트로폴리스
13366,쫄라맨,마틴,서포터,활성화 지원,신경쇠약,부서진 갑옷,E 마스터 마인드,유니크,사이트 인베스트,레어,S 루브르 디텍터,유니크,S 메즈메리즘,유니크,E 솔로스 인듀서,유니크,E 데인저러스 플롯,유니크,파이널 레그람,레어,힙노시스 스팟,레어,첼시 콜라 MAX,레어,런업 스프린터,레어,마인드 이펙트,레어,솔리드 스위퍼,레어,최면 임팩트,언커먼,E 힙노티스트,유니크,마리오넷 레그람,레어,사일런트 레그람,레어,False,56,2,4,25,27715,23355,346,563,16980,4400,10,12,1260,106,495,102,1,7e5c628d939498bc3ee75fb96026f9aef96197ce71da93...,메트로폴리스
13367,신라면순한맛,휴톤,탱커,전투 태세,단단한 피부,최후의 저항,SP혼신의 일격,레어,테너서티 컨트롤,레어,DB본 헤르츠,레어,SP플래티넘 체인,레어,SP풋워크 트릭,레어,FA락포트 버스터,레어,지옥구멍 레그람,레어,바야바 레그람,레어,본 스파클링,레어,런업 스프린터,레어,아이리쉬 이펙트,레어,아이리쉬 플래쉬,레어,인사이트 엔진T7,언커먼,플래티넘 체인,레어,서든어퍼 레그람,레어,누크 퓨리피스트,유니크,False,56,4,6,17,27126,35166,295,433,15740,3000,10,8,1260,45,493,131,1,7e5c628d939498bc3ee75fb96026f9aef96197ce71da93...,메트로폴리스
13368,서해준,엘프리데,원거리딜러,재장전,전장의 학살자,전장의 열기,시크 브레이슬릿,레어,S 감춰진 시선,유니크,E 오버 필링,유니크,E 레스트레인트,유니크,S 완벽한 자유,유니크,S 온화한 발걸음,유니크,팰리스 메모,레어,저스티스 페인트북,레어,체스터 더블버거,레어,엑셀레이션,레어,이펙